In [1]:
import pandas as pd   
import requests 
import os 
import time
from dotenv import load_dotenv

load_dotenv()



True

In [2]:
dataset = pd.read_excel('./prompt_library.xlsx')


In [3]:
dataset.head()

,Prompt
0,Are there any collaborative works between Micr...
1,Assess the potential impact of [Project Name] ...
2,Can we draw any parallels between Microsoft's ...
3,Can we identify any new patterns in the applic...
4,Can you help me understand the technical aspec...


In [4]:
def get_response(question, response=None):
    """
    Sends a POST request to the conversation API to obtain a response based on the given user question and optionally response from a previous question.

    Parameters:
    - question (str): The user's question to be sent to the conversation API from the excel file.
    - response (dict or None): The optional AI response to be included in the conversation. If None, only the user's question is sent.

    Environment Variables:
    - PROMPT_EVAL_STRING: Evaluation string to be included in the user's messages which will be used to prompt the AI to evaluate its previous response.
    - BASE_URL (required): The Base URL of the API. If not provided, defaults to "http://127.0.0.1:5000".
    
    """
    evaluation_string = os.environ.get("PROMPT_EVAL_STRING")

    base_url = os.environ.get("BASE_URL", "http://127.0.0.1:5000") 
    url = f'{base_url}/conversation'
    headers = {
        "content-type": "application/json",
    }
    try:
        if response:
            data = {
                "messages": [
                    {
                        "role": "user",
                        "content": question,
                    },
                    {
                        "role": "tool",
                        "content": response["choices"][0]["messages"][0]["content"],
                    },
                    {
                        "role": "assistant",
                        "content": response["choices"][0]["messages"][1]["content"],
                    },
                    {"role": "user", "content": evaluation_string},
                ]
            }
        else:
            data = {"messages": [{"role": "user", "content": question}]}
        response = requests.post(url, json=data, headers=headers)
        if not response.status_code == 200:
            print("POST request failed. Status code:", response.status_code)
            raise Exception

        response_json = response.json()

        return response_json

    except (KeyError, IndexError) as e:
        raise e

In [5]:
# Function to rate the answer
def evaluate_response(response, question):
  """
    Rate the quality of an AI-generated response based on the provided response and original user question.

    Parameters:
    - response (dict): The JSON response obtained from the conversation API, containing AI-generated response.
    - question (str): The original user question.
  """
  response = get_response(question=question, response=response)
  evaluation = response["choices"][0]["messages"][1]["content"]
  return evaluation

In [6]:
def process_row(row):
    """
    Process a row of data containing a user question and obtain the AI-generated response along with an evaluation response/rating.

    Parameters:
    - row (dict): A dictionary representing a row of data, typically from a DataFrame, containing a 'Prompt' field.

    """
    question = row['Prompt']
    start_time = time.time()
    response = get_response(question=question)
    end_time = time.time()
    answer  = response["choices"][0]["messages"][1]["content"]
    evaluation = evaluate_response(response=response, question=question)
    response_time = end_time - start_time
    return question, answer, evaluation, response_time

In [ ]:
# Process each row and store the results in a new DataFrame
results = []
for index, row in dataset.iterrows():
    try:
        results.append(process_row(row))
        print(f"Question {index} processed succesfully")
    except Exception as e:
        print(f"Error Question {index} processed unsuccesfully")
        continue

In [8]:
# Create a new DataFrame from the results
output_dataset = pd.DataFrame(results, columns=['Question', 'Answer', 'Evaluation', 'Response Time (s)'])

In [9]:
# Write the new DataFrame to a new Excel file
output_file_path = 'output_data.xlsx'
output_dataset.to_excel(output_file_path, index=False)

In [10]:
output_dataset

,Question,Answer,Evaluation,Response Time (s)
0,Are there any collaborative works between Micr...,"Yes, there are collaborative works between Mic...",The response provided in the previous interact...,60.826001
1,Assess the potential impact of [Project Name] ...,The requested information is not available in ...,The user query is a request to assess the qual...,21.279244
2,Can we draw any parallels between Microsoft's ...,The retrieved documents provide insights into ...,The response provided does not align with the ...,101.887679
3,Can we identify any new patterns in the applic...,The requested information is not available in ...,The response provided is in line with the inst...,16.669292
4,Can you help me understand the technical aspec...,The requested information is not available in ...,The response provided to the user's initial qu...,18.022013
...,...,...,...,...
135,What technical innovations has MSR introduced ...,Microsoft Research (MSR) has introduced severa...,The response provided by the assistant does no...,88.390844
136,How is MSR's research tackling the challenge o...,Microsoft Research (MSR) is addressing the cha...,The response provided to the user's question a...,54.146109
137,What are the cutting-edge techniques being use...,The retrieved documents highlight several cutt...,The response provided cannot be evaluated for ...,65.400857
138,What kind of problems is MSR's AI research try...,Microsoft Research (MSR) is actively working o...,The response provided is comprehensive and wel...,55.280129


In [11]:
print(f"Processing complete. Results saved to {output_file_path}")

Processing complete. Results saved to output_data.xlsx
